In [3]:
!pip install tensorflow

In [4]:
import tensorflow

2.13.0


In [5]:
!pip install mtcnn

     ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
      --------------------------------------- 0.0/2.3 MB 219.4 kB/s eta 0:00:11
     - -------------------------------------- 0.1/2.3 MB 454.0 kB/s eta 0:00:05
     -- ------------------------------------- 0.2/2.3 MB 656.4 kB/s eta 0:00:04
     ---- ----------------------------------- 0.2/2.3 MB 838.1 kB/s eta 0:00:03
     ----- ---------------------------------- 0.3/2.3 MB 925.5 kB/s eta 0:00:03
     ------ --------------------------------- 0.4/2.3 MB 1.0 MB/s eta 0:00:02
     -------- ------------------------------- 0.5/2.3 MB 1.1 MB/s eta 0:00:02
     --------- ------------------------------ 0.5/2.3 MB 1.1 MB/s eta 0:00:02
     ---------- ----------------------------- 0.6/2.3 MB 1.2 MB/s eta 0:00:02
    

In [7]:
import os
import numpy as np
import cv2

from PIL import Image
from numpy import asarray
from mtcnn.mtcnn import MTCNN
from keras.models import load_model,model_from_json

In [12]:
model_path = "Models/Inception_ResNet_v1.json"
weights_path = "Models/facenet_keras_weights.h5"

json_file = open(model_path, 'r')
loaded_model_json = json_file.read()
json_file.close()
enc_model = model_from_json(loaded_model_json)
enc_model.load_weights(weights_path)

In [ ]:
mtcnn_detector = MTCNN()

In [ ]:
def detect_face(filename, required_size=(160, 160),normalize = True):

    img = Image.open(filename)

    img = img.convert('RGB')
 
     pixels = np.asarray(img)
 
     results = mtcnn_detector.detect_faces(pixels)
 
    x1, y1, width, height = results[0]['box']

    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height

    face = pixels[y1:y2, x1:x2]
  
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = asarray(image)
 
    if normalize == True:

        mean = np.mean(face_array, axis=(0,1,2), keepdims=True)
        std = np.std(face_array, axis=(0,1,2), keepdims=True)
        std_adj = np.maximum(std, 1.0)
        return (face_array - mean) / std

    else : 
        return face_array

In [ ]:
known_faces_encodings = []
known_faces_ids = []

known_faces_path = "Face_database/"

for filename in os.listdir(known_faces_path):
  
  face = detect_face(known_faces_path+filename,normalize = True)

  feature_vector = enc_model.predict(face.reshape(1,160,160,3))
  feature_vector/= np.sqrt(np.sum(feature_vector**2))
  known_faces_encodings.append(feature_vector)
  label = filename.split('.')[0]
  known_faces_ids.append(label)


known_faces_encodings = np.array(known_faces_encodings).reshape(len(known_faces_encodings),128)
known_faces_ids = np.array(known_faces_ids)

In [ ]:
print(known_faces_ids.shape[0])

In [ ]:

def recognize(img,known_faces_encodings,known_faces_ids,threshold = 0.75):

  scores = np.zeros((len(known_faces_ids),1),dtype=float)

  enc = enc_model.predict(img.reshape(1,160,160,3))
  enc/= np.sqrt(np.sum(enc**2))

  scores = np.sqrt(np.sum((enc-known_faces_encodings)**2,axis=1))

  match = np.argmin(scores)

  if scores[match] > threshold :

    return ("UNKNOWN",0)
      
  else :

    return (known_faces_ids[match],scores[match])

In [ ]:

def face_recognition(mode,file_path,known_faces_encodings,known_faces_ids,
                         detector = 'haar', threshold = 0.75):

  if detector == 'haar':

    face_cascade = cv2.CascadeClassifier('Models/haarcascade_frontalface_default.xml')

  if mode == 'webcam':

    cap = cv2.VideoCapture(0)

  elif mode == 'video':
    cap = cv2.VideoCapture(file_path)
    
  while True:

    _, img = cap.read()
    
=    if _ == False:
        break;

    if detector == 'haar':
    
      gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

      faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    elif detector == 'mtcnn' :  

      results = mtcnn_detector.detect_faces(img)
      
      if(len(results)==0):
        continue

      faces = []
    
      for i in range(len(results)):
        
        x,y,w,h = results[i]['box']
        x, y = abs(x), abs(y)
        faces.append([x,y,w,h])

    for (x, y, w, h) in faces:
        
        image = Image.fromarray(img[y:y+h, x:x+w])
        image = image.resize((160,160))
        face_array = asarray(image)

        mean = np.mean(face_array, axis=(0,1,2), keepdims=True)
        std = np.std(face_array, axis=(0,1,2), keepdims=True)
        std_adj = np.maximum(std, 1.0)
        face_array_normalized = (face_array - mean) / std

        label = recognize(face_array_normalized,known_faces_encodings,known_faces_ids,threshold = 0.75)
        
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 0, 0), 2)
        cv2.putText(img, label[0], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,255,255), 2)

    cv2.imshow('Face_Recognition', img)
    
    key = cv2.waitKey(25) & 0xff
    if key==27:
        break

  cap.release() 

In [ ]:

face_recognition('webcam',None,known_faces_encodings,known_faces_ids,
                 detector = 'haar',threshold = 0.75)

In [ ]:

face_recognition('video',"test.mp4",known_faces_encodings,known_faces_ids,
                 detector = 'mtcnn',threshold = 0.75)